In [1]:
import pickle
import numpy as np
import networkx as nx
import pandas as pd
import brainmaptools as brainmap
import matplotlib.pyplot as plt
import community
import brainx.modularity as mod
import operator
%matplotlib inline
import csv
import scipy as scipy
import scipy.io as io
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist
import math as math
import pylab as pl
import My_functions as myfunctions
import progressbar as pb
import time
from collections import Counter
import random
import os
%load_ext Cython
import bct
from numba import jit

/Users/owner/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/brainx/__init__.py:30: UserWarning: Monkeypatching NetworkX's Watts-Strogatz routine
  warnings.warn("Monkeypatching NetworkX's Watts-Strogatz routine")


In [ ]:
domains= ['Memory', 'Working Memory', 'Emotion', 'Attention', 'Language', 'Vision', 'Audition']
domainsfile="/Users/owner/Functional_Connectivity/Results/studies_filtered_by_domain.pck"
domaindata=pickle.load(open(domainsfile, "rb" ))

In [ ]:
key_codes=brainmap.build_key_codes_from_workspaces(workspaces, datadir)
domain_filtered_keycodes=dict()
for x in domains:
    domain_filtered_keycodes[x]=brainmap.domain_filter_keycodes(key_codes,domaindata,x)

In [ ]:
domains.append('Base')
domain_filtered_keycodes['Base']=key_codes

In [ ]:
relabel_dict_file='/Users/owner/Functional_Connectivity/Scripts/Label_dict.pck'
relabel_dict=pickle.load(open(relabel_dict_file,"rb"))

In [ ]:
G_nothresh={}
for x in domains:
    filename='G_nothresh_jaccard_'+x+'_.pck'
    workdir='/Users/owner/Functional_Connectivity/Results/G_jaccard_graphs/Unthresholded_G_jaccard/'
    output=workdir+filename
    G_nothresh[x]=pickle.load(open (output, 'rb'))

# Create Influence Graphs

In [ ]:
influence_G={}
influence_matrix={}
for x in domains: 
    influence_matrix[x]=brainmap.build_influence_digraph(brainmap.build_n_coactives_array(domain_filtered_keycodes[x]))
    influence_G[x]=nx.relabel_nodes(influence_matrix[x],relabel_dict)

In [ ]:
directed_edge=pd.DataFrame(nx.to_numpy_matrix(influence_G['Base']))
directed_edge.to_csv('/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/EdgeFiles/Influence/CSV/Influence_edge.csv', header=False, index=False, index_label=False)
csv_file = '/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/EdgeFiles/Influence/CSV/Influence_edge.csv'
txt_file = '/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/EdgeFiles/Influence/Edges/Influence_edge.edge'
with open(txt_file, "w") as my_output_file:
    with open(csv_file, "r") as my_input_file:
        [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [ ]:
number_influencers={node: len(influence_G['Base'].predecessors(node)) for node in influence_G['Base'].nodes()}

In [ ]:
number_successors={node: len(influence_G['Base'].successors(node)) for node in influence_G['Base'].nodes()}

In [ ]:
compiled_list=[]
for region in relabel_dict.values():
    compiled_list.append(region[:-1])
compiled_list=set(compiled_list)
compiled_list=[x for x in compiled_list if not any(c.isdigit() for c in x)]
compiled_list=sorted(compiled_list)

In [ ]:
compiled_region_dict={}
for substring in compiled_list:
    for region in relabel_dict.values():
        if substring in region:
            compiled_region_dict.setdefault(substring, []).append(region)

In [ ]:
successor_dict={node: influence_G['Base'].successors(node) for node in influence_G['Base'].nodes()}

In [ ]:
for influencer in successor_dict.keys():
    successor_dict[influencer].append(influencer)

In [ ]:
parent_region_dict={}
for key in compiled_region_dict.keys():
    for subregion in successor_dict.keys():
        if key in subregion:
            parent_region_dict.setdefault(key, []).append(successor_dict[subregion])

In [ ]:
parent_final ={parent: [item for sublist in parent_region_dict[parent] for item in sublist] for parent in parent_region_dict.keys()}

In [ ]:
subnetwork={parent: influence_G['Base'].subgraph(set(parent_final[parent])) for parent in parent_final.keys()}

In [ ]:
for parent in compiled_region_dict.keys():
    for edge in subnetwork[parent].edges():
        if parent not in edge[0] and parent not in edge[1]:
            subnetwork[parent].remove_edge(edge[0], edge[1])
            brainmap.remove_edgeless_nodes(subnetwork[parent])
        for subregion in subnetwork[parent].nodes():
            if parent in subregion:
                for pred in subnetwork[parent].predecessors(subregion):
                    subnetwork[parent].remove_edge(pred, subregion)
                    brainmap.remove_edgeless_nodes(subnetwork[parent])

In [ ]:
Create Version Put Through Threshold

In [ ]:
thresh=[0.06, 0.07, 0.08, 0.09, 0.1]

In [ ]:
subnet_cost={parent: {cost: brainmap.applycost_to_g(influence_G['Base'], cost) for cost in thresh} for parent in subnetwork.keys()}

In [ ]:
for parent in subnet_cost.keys():
    for cost in subnet_cost[parent].keys():
        for edge in subnet_cost[parent][cost].edges():
            if parent not in edge[0] and parent not in edge[1]:
                subnet_cost[parent][cost].remove_edge(edge[0], edge[1])
                brainmap.remove_edgeless_nodes(subnet_cost[parent][cost])
            for subregion in subnet_cost[parent][cost].nodes():
                if parent in subregion:
                    for pred in subnet_cost[parent][cost].predecessors(subregion):
                        subnet_cost[parent][cost].remove_edge(pred, subregion)
                        brainmap.remove_edgeless_nodes(subnet_cost[parent][cost])

In [ ]:
dataf=pd.read_csv('/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/NodeFiles/CSV/Label_coordinates.csv', header=None, names=['x','y','z','regions'])
dataf.insert(4,'type',0,True)
dataf.insert(5,'size',0,True)
dataf.insert(6,'nodes',0,True)
dataf=dataf[['x','y','z','type','size','regions','nodes']]

In [ ]:
influencedf={parent: {cost: pd.read_csv('/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/NodeFiles/CSV/Label_coordinates.csv', header=None, names=['x','y','z','regions']) for cost in subnet_cost[parent].keys()} for parent in subnet_cost.keys()}
sortnodes={parent: {cost: dict(zip(subnet_cost[parent][cost].nodes(),range(subnet_cost[parent][cost].number_of_nodes()))) for cost in subnet_cost[parent].keys()} for parent in subnet_cost.keys()}
for parent in subnet_cost.keys():
    for cost in subnet_cost[parent].keys():
        influencedf[parent][cost]=pd.read_csv('/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/NodeFiles/CSV/Label_coordinates.csv', header=None, names=['x','y','z','regions'])
        influencedf[parent][cost].insert(4,'type',0,True)
        influencedf[parent][cost].insert(5,'size',0,True)
        influencedf[parent][cost].insert(6,'nodes',0,True)
        influencedf[parent][cost]=influencedf[parent][cost][['x','y','z','type','size','regions','nodes']]
        for region in subnet_cost[parent][cost].nodes():
            for n in range(len(influencedf[parent][cost])):
                if region == influencedf[parent][cost].loc[n,'regions']:
                    influencedf[parent][cost].loc[n,'nodes']=region
                    influencedf[parent][cost].loc[n, 'size']=0
                    if parent in region:
                        influencedf[parent][cost].loc[n, 'type']=1
        influencedf[parent][cost]=influencedf[parent][cost][['x','y','z','type','size','nodes']]
        influencedf[parent][cost]=influencedf[parent][cost][influencedf[parent][cost].nodes != 0]
        influencedf[parent][cost]['sorted_nodes']=influencedf[parent][cost]['nodes'].map(sortnodes[parent][cost])
        influencedf[parent][cost].sort_values(['sorted_nodes'], ascending=True, inplace=True)
        influencedf[parent][cost].drop('sorted_nodes', 1, inplace=True)
        influencedf[parent][cost].index=range(len(influencedf[parent][cost]))
        outdir='/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/NodeFiles/Influence/CSV/'
        if influencedf[parent][cost].empty == True:
            print ('{} Dataframe at cost {} is empty.'.format(parent, cost))
        else:
            filename=parent+str(cost)+'_Influences.csv'
            influencedf[parent][cost].to_csv(outdir+filename, header=False, index=False, index_label=False)

Create BrainNet File of Influence Graph

In [ ]:
for parent in subnet_cost.keys():
    for cost in subnet_cost[parent].keys():
        csv_file ='/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/NodeFiles/Influence/CSV/'+parent+str(cost)+'_Influences.csv'
        if os.path.isfile(csv_file):
            txt_file = '/Users/owner/Desktop/BrainImagingViewer/BrainNetViewer_20171031/Data/NodeFiles/Influence/'+parent+str(cost)+'_Influences.node'
            with open(txt_file, "w") as my_output_file:
                with open(csv_file, "r") as my_input_file:
                    [ my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
                my_output_file.close()